In [ ]:
import os
os.environ['OPENAI_API_KEY'] = ''
from openai import OpenAI

client = OpenAI()

In [ ]:
# upload jsonl files
jsonl_path = ""  # the folder path of your jsonl files
for jsonl in os.listdir(jsonl_path):
    if jsonl.endswith('.jsonl'):
      # 上传文件
      batch_input_file = client.files.create(
        file=open(os.path.join(jsonl_path, jsonl), "rb"),
        purpose="batch"
      )
      batch_input_file_id = batch_input_file.id

      # 创建进程
      client.batches.create(
          input_file_id=batch_input_file_id,
          endpoint="/v1/chat/completions",
          completion_window="24h",
          metadata={
            "description": f"{jsonl}"
          }
      )
      print(f"{jsonl} is created.")

batchinput-00041.jsonl is created.
batchinput-00042.jsonl is created.
batchinput-00043.jsonl is created.
batchinput-00044.jsonl is created.
batchinput-00045.jsonl is created.
batchinput-00046.jsonl is created.


In [ ]:
# check the status of your uploaded batches

for batch in client.batches.list().data:
    print(batch)
    print(f"{batch.metadata['description']} is {batch.status}")

Batch(id='batch_683efd82fcbc8190be202b1e2e1170dc', completion_window='24h', created_at=1748958594, endpoint='/v1/chat/completions', input_file_id='file-EpEKUWYaMHwrs27SjgDuFs', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1748959218, error_file_id=None, errors=None, expired_at=None, expires_at=1749044994, failed_at=None, finalizing_at=1748959200, in_progress_at=1748958596, metadata={'description': 'batchinput-00000.jsonl'}, output_file_id='file-Jx45mUERmnQU2T2QjP9Yxx', request_counts=BatchRequestCounts(completed=50, failed=0, total=50))
batchinput-00000.jsonl is completed


In [ ]:
# download the output files of completed batches

output_path = ""
if not os.path.exists(output_path):
    os.makedirs(output_path)
for batch in client.batches.list(limit=1).data:
    if batch.status == "completed" and batch.output_file_id is not None:
        if f"{batch.metadata['description']}" in os.listdir(output_path): 
            continue
        else:
            content = client.files.content(batch.output_file_id)
            dict_strings = content.text.strip().split('\n')
            
            save_path = f'{output_path}/{batch.metadata['description']}' 
            with open(save_path, 'w') as jsonl_file:
                for dict_string in dict_strings:
                    jsonl_file.write(dict_string + '\n')
            print(f"{batch.metadata['description']} is downloaded.")
    # else:
    #     print(f"{batch.id} is {batch.status}.")


batchinput-00000.jsonl is downloaded.
